<h1>Template Matching</h1>

In [1]:
import cv2 as cv
import numpy as np

# Load the input image and the template image (heart)
card_img = cv.imread(r"C:\Users\vamsh\Desktop\NLP Images\Card.png")
template = cv.imread(r"C:\Users\vamsh\Desktop\NLP Images\heart.png")

In [2]:
# Convert both images to grayscale
gray_image = cv.cvtColor(card_img, cv.COLOR_BGR2GRAY)
gray_template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)


In [3]:
# Set a lower threshold for matching
threshold = 0.6 #Lowered threshold for partial matches


In [5]:
# Perform template matching at multiple scales
matches_found = False
all_locations = []

# Get the dimensions of the original image
img_height, img_width = gray_image.shape

# Try matching template with different sizes or scales
for scale in np.arange(0.5, 1.5, 0.1):   #Scale from 50% to 150%
    resized_template = cv.resize(gray_template, (0,0), fx=scale, fy=scale)
    w, h = resized_template.shape[::-1]

    # Check if the template size is larger than the image size at the current size at the current
    if w > img_width or h > img_height:
        continue  #Skips the current iteration and moves to the next scale

    result = cv.matchTemplate(gray_image, resized_template, cv.TM_CCOEFF_NORMED)

    # Find locations where the match is greater than the threshold
    locations = np.where(result >= threshold)

    # This loop extracts and stores the detected template match locations of template height and width
    for pt in zip(*locations[::-1]): #Reverse the order of locations
        all_locations.append((pt[0], pt[1], w, h))

# all_locations - stores all the detected template positions
if len(all_locations) > 0:
    boxes = np.array(all_locations)
    x_coords = boxes[:, 0] #Extract top-left x-coordinate of matched template
    y_coords = boxes[:, 1] #Extract top-left y-coordinates
    widths = boxes[:, 2] #Width of each template
    heights = boxes[:, 3] #Height of each template

    # Compute the bottom-right x and y coordinate
    x2_coords = x_coords + widths
    y2_coords = y_coords + heights

    # This combines all four arrays (x1, y1, x2, y2) into a single NumPy array and transpose to draw bounding box
    boxes = np.array([x_coords, y_coords, x2_coords, y2_coords]).T

    # Perform NMS (non Max supression) using OpenCV function
    # NMS- helps to remove unnecessary overlapping boxes
    # [1]*len(boxes) - compute configure score, 1-dummy score, len(boxes) - no.
    indices = cv.dnn.NMSBoxes(boxes.tolist(), [1]*len(boxes), score_threshold=0.6, nms_threshold=0.3)

    # Draw the bounding boxes
    if len(indices) > 0:
        for i in indices.flatten():
            x1, y1, x2, y2 = boxes[i]
            cv.rectangle(card_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        matches_found = True
    
                            

In [7]:
# Display the result with bounding boxes
cv.imshow('Detected Hearts on Original Image', card_img)
cv.waitKey(0)
cv.destroyAllWindows()